In [ ]:
!pip install youtube_dl --upgrade


In [ ]:
import sys
from youtube_dl import YoutubeDL
path="./"
def video_download():
    file = open(path+"carcamvideoslist.txt",'r')
    urllist=file.readlines()
    print(urllist)
    if len(urllist)>0:
        ydl_opts = {}
        ydl = YoutubeDL(ydl_opts)
        ydl.download(urllist)
    else:
        print("Enter list of urls to download")
        exit(0)

In [ ]:
video_download()

In [2]:
# derive the paths to the YOLO weights and model configuration
weightsPath = "C:\\Users\\User\\\OpenCV tutorial\\yolo-coco\\yolov3.weights"
configPath = "C:\\Users\\User\\OpenCV tutorial\\yolo-coco\\yolov3.cfg"
labelsPath = "C:\\Users\\User\\\OpenCV tutorial\\yolo-coco\\coco.names"
    

In [3]:
#some of the codes is re-use from pyimagesearch, thanks to Adrian Rosebrock
#https://www.pyimagesearch.com/
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
import time
import math

In [13]:
def ProcessVideo(videoFile,conf,thres,weightsPath,configPath,labelsPath):
    import json
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    np.random.seed(42)
    # initialize a list of colors to represent each possible class label
    LABELS = open(labelsPath).read().strip().split("\n")
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")
    
    # initialize the video stream, pointer to output video file, and
    # frame dimensions
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    vs = cv2.VideoCapture(videoFile)
    writer = None
    (W, H) = (None, None)
    try:
        prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
            else cv2.CAP_PROP_FRAME_COUNT
        total = int(vs.get(prop))
        print("[INFO] {} total frames in video".format(total))

    # an error occurred while trying to determine the total
    # number of frames in the video file
    except:
        print("[INFO] could not determine # of frames in video")
        print("[INFO] no approx. completion time can be provided")
        total = -1
    frame_no=0
    videoDetails={}
    with open(videoFile+'.json', 'w') as fp:
        # loop over frames from the video file stream
        while True:
            # read the next frame from the file
            (grabbed, frame) = vs.read()
            frame_no=frame_no+1
            # if the frame was not grabbed, then we have reached the end
            # of the stream
            if not grabbed:
                break

            # if the frame dimensions are empty, grab them
            if W is None or H is None:
                (H, W) = frame.shape[:2]

            # construct a blob from the input frame and then perform a forward
            # pass of the YOLO object detector, giving us our bounding boxes
            # and associated probabilities
            blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                swapRB=True, crop=False)
            net.setInput(blob)
            start = time.time()
            layerOutputs = net.forward(ln)
            end = time.time()

            # initialize our lists of detected bounding boxes, confidences,
            # and class IDs, respectively
            boxes = []
            confidences = []
            classIDs = []

            # loop over each of the layer outputs
            for output in layerOutputs:
                # loop over each of the detections
                for detection in output:
                    # extract the class ID and confidence (i.e., probability)
                    # of the current object detection
                    scores = detection[5:]
                    classID = np.argmax(scores)
                    confidence = scores[classID]

                    # filter out weak predictions by ensuring the detected
                    # probability is greater than the minimum probability
                    if confidence > conf:
                        # scale the bounding box coordinates back relative to
                        # the size of the image, keeping in mind that YOLO
                        # actually returns the center (x, y)-coordinates of
                        # the bounding box followed by the boxes' width and
                        # height
                        box = detection[0:4] * np.array([W, H, W, H])
                        (centerX, centerY, width, height) = box.astype("int")

                        # use the center (x, y)-coordinates to derive the top
                        # and and left corner of the bounding box
                        x = int(centerX - (width / 2))
                        y = int(centerY - (height / 2))

                        # update our list of bounding box coordinates,
                        # confidences, and class IDs
                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))
                        classIDs.append(classID)

            # apply non-maxima suppression to suppress weak, overlapping
            # bounding boxes
            idxs = cv2.dnn.NMSBoxes(boxes, confidences, conf,
                thres)
            objectDimensions={}
            # ensure at least one detection exists
            if len(idxs) > 0:
                # loop over the indexes we are keeping
                for i in idxs.flatten():
                    # extract the bounding box coordinates
                    (x, y) = (boxes[i][0], boxes[i][1])
                    (w, h) = (boxes[i][2], boxes[i][3])

                # draw a bounding box rectangle and label on the frame
                    color = [int(c) for c in COLORS[classIDs[i]]]
                    DistanceFromLeftCenter= math.sqrt(math.pow((x+w/2)-W/4,2)+ math.pow((y+h/2)-H/2,2))
                    DistanceFromRightCenter=math.sqrt(math.pow((x+w/2)-3*W/4,2)+ math.pow((y+h/2)-H/2,2))
                    collisionScore= math.sqrt(len(idxs)*((3*w*w/W)+(h*h/H))/(DistanceFromLeftCenter+DistanceFromRightCenter))
                    if collisionScore >0.4:
                        
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        text = "{} W2HR:{:.4f} colliScr:{:.4f}".format(LABELS[classIDs[i]],w/h,collisionScore)
                        cv2.putText(frame, text, (x, y - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    objectDimensions[i]=[w,h,(x+w/2),(y+h/2),LABELS[classIDs[i]],confidences[i]]
                
                if bool(objectDimensions)==True:
                    fp.write(str(frame_no)+":"+str(objectDimensions)+",")
            # check if the video writer is None
            if writer is None:
                # initialize our video writer
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter(videoFile.split(".")[0]+"_YOLO.avi", fourcc, 30,
                    (frame.shape[1], frame.shape[0]), True)

                # some information on processing single frame
                if total > 0:
                    elap = (end - start)
                    print("[INFO] single frame took {:.4f} seconds".format(elap))
                    print("[INFO] estimated total time to finish: {:.4f}".format(
                        elap * total))

            # write the output frame to disk
            writer.write(frame)
    


In [14]:
ProcessVideo("source.mp4",0.5,0.2,weightsPath,configPath,labelsPath)

[INFO] 900 total frames in video
[INFO] single frame took 0.6742 seconds
[INFO] estimated total time to finish: 606.7517
